In [10]:
import numpy as np
import glob2 as glob
import pandas as pd

In [25]:
files = glob.glob('2K_model_weights_3_layers/*.npy')

weights_dict = {'bias' : {}, 'weight' : {}}

for file in files:
    layer_name = file.split('/')[-1].split('.')[0]
    if layer_name.split('_')[-1] == 'bias':
        weights_dict['bias'].update({layer_name.split('_')[0]: np.load(file)})
    else:
        weights_dict['weight'].update({layer_name.split('_')[0]: np.load(file)})

test_df = pd.read_csv('mnist_test.csv')
order= ['fc1', 'fc2', 'fc3']

In [74]:
def predict(img_df_row):

    x = np.dot(weights_dict['weight']['fc1'], img_df_row.T) + weights_dict['bias']['fc1']
    x = np.maximum(0, x)

    x = np.dot(weights_dict['weight']['fc2'], x.T) + weights_dict['bias']['fc2']
    x = np.maximum(0, x)

    x = np.dot(weights_dict['weight']['fc3'], x.T) + weights_dict['bias']['fc3']
    x = np.maximum(0, x)

    x /= np.sum(x)

    return x

predict(test_df.drop(columns=['label']).iloc[3])

array([0.95830522, 0.        , 0.04169478, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [46]:
test_df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
